In [115]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [116]:
# get adj_matrix using given file
# return adj_matrix
def generate_adjacency_matrix(file_name):
    input_graph = np.loadtxt(file_name)
    input_graph = input_graph.astype(int)-1
    size = max(max(input_graph[:,0]),max(input_graph[:,1]))+1
    adjacency_matrix = np.zeros((size,size))
    for pair in input_graph:
        adjacency_matrix[pair[0]][pair[1]] = 1
        adjacency_matrix[pair[1]][pair[0]] = 1
    return adjacency_matrix

In [117]:
class Max_k_DSBG:
    def __init__(self, adjacency_matrix):
        self.adjacency_matrix = adjacency_matrix
        self.num_vertex = adjacency_matrix.shape[0]
        self.num_edge = None
        self.incidence_matrix = None
        self.edge_list = []
        self.k = None
        self.S = None

    # determine whether the input graph is bipartite, if not, output promopt information. If it is, asking for a input k
    def is_bipartite(self):
        size = self.num_vertex
        color_arr = [-1]*size
        color_arr[0] = 1
        queue = []
        queue.append(0)

        while queue:
            u = queue.pop()
            
            for v in range(size):
                if self.adjacency_matrix[u][v] == 1 and color_arr[v] == -1:
                    color_arr[v] = 1-color_arr[u]
                    queue.append(v)
                elif self.adjacency_matrix[u][v] == 1 and color_arr[v] == color_arr[u]:
                    return print("The input graph is NOT bipartite")
        self.k = int(input("value of k")) 

    # generate an incidence matrix
    def get_incidence_matrix(self):
        size = self.num_vertex
        index = 0
        for i in range(size):
            for j in range(i+1, size):
                if self.adjacency_matrix[i][j]:
                    self.edge_list.append((i, j,index))
                    index += 1
        self.num_edge = len(self.edge_list)
        self.incidence_matrix = np.zeros((self.num_vertex,self.num_edge))
        for element in self.edge_list:
            self.incidence_matrix[element[0]][element[2]] = 1
            self.incidence_matrix[element[1]][element[2]] = 1

    # solve maximum independent matching problem using gurobi
    def maximum_independent_matching(self):
        edge_result = []
        model = gp.Model("maximum bipartite matching problem")
        x = model.addMVar(shape = self.num_edge, lb=0, vtype = GRB.BINARY)
        model.setObjective(x.sum(),GRB.MAXIMIZE)
        model.addConstr(self.incidence_matrix @ x <= 1)

        model.Params.OutputFlag = 0
        model.optimize()

        for var in model.getVars():
            edge_result.append(var.x)
        '''print("Maximum bipartite matching: ", model.ObjVal)
        print("matching: ")
        print(edge_result)'''

        return edge_result


    # remove matching we've found in our current graph and update graph
    def remove_matching(self):
        edge_result = self.maximum_independent_matching()

        if len(edge_result) == 0:
            return 

        delete_index = []
        for i in range(len(edge_result)):
            if edge_result[i] == 1:
                delete_index.append(i)
                self.num_edge -= 1

        self.incidence_matrix = np.delete(self.incidence_matrix,delete_index,axis=1)
 
        '''print("after removing")
        print(self.incidence_matrix)
        print(self.num_edge)
        print()'''

    # solve minimum vertex cover problem using gurobi
    def minimum_vertex_cover_problem(self):
        vertex_result = []
        model = gp.Model("minimum vertex cover problem")
        y = model.addVars(self.num_vertex, lb=0, vtype = GRB.BINARY)
        model.setObjective(y.sum(), GRB.MINIMIZE)
        for i in range(self.num_edge):
            edge = self.incidence_matrix[:,i]
            if np.count_nonzero(edge) == 0:
                continue
            non_zero = np.nonzero(edge)
            model.addConstr(y[non_zero[0][0]]+y[non_zero[0][1]]>=1)    
            
        model.Params.OutputFlag = 0
        model.optimize()

        for var in model.getVars():
            vertex_result.append(var.x)
        # print("Minimum bipartite vertex cover: ", model.ObjVal)
        return vertex_result

    # sovle maximum k dependent set   
    def maximum_k_dependent_set(self):
        size = self.num_vertex
        for i in range(1, self.k+1):
            self.remove_matching()
        
        vertex_result = self.minimum_vertex_cover_problem()

        # maximum independent set S in the residual graph
        self.S = [0]*size
        for i in range(len(vertex_result)):
            if vertex_result[i]:
                self.S[i] = 0
            else:
                self.S[i] = 1
    
    # output the vertex in maximum k dependent set and the cardinality of it
    def output(self):
        print("Maximum k-dependent set:", end = "\n")
        total = 0
        for i in range(len(self.S)):
            if self.S[i]:
                print("vertex{}".format(i+1), end = "  ")
                total += 1
        print()
        print()
        print("Maximum cardinality of k-dependent set in G: ", total)
        print("----------------------------------------------------------------------------------------------------")

        return total

In [118]:
def main():
    adjacency_matrix = generate_adjacency_matrix("test_1.txt")
    k_DSBG = Max_k_DSBG(adjacency_matrix)
    k_DSBG.is_bipartite()
    k_DSBG.get_incidence_matrix()
    k_DSBG.maximum_k_dependent_set()
    k_DSBG.output()

main()


Maximum k-dependent set:
vertex2  vertex3  vertex4  vertex5  vertex6  vertex7  vertex8  vertex9  vertex10  vertex11  vertex12  vertex13  vertex14  vertex15  vertex16  vertex20  vertex22  vertex23  vertex24  vertex26  vertex27  vertex29  vertex30  vertex31  vertex33  vertex35  

Maximum cardinality of k-dependent set in G:  26
----------------------------------------------------------------------------------------------------
